<a href="https://colab.research.google.com/github/Guzzler/IDL_Project_Audio_Segmentation/blob/main/SoundDescs_Data_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the BBC Sound Effects Dataset
Follow instructions at https://github.com/akoepke/audio-retrieval-benchmark

## Clip Audios

In [ ]:
# Requirements
!pip install wget zsvision pydub

In [ ]:
%cd "/content/drive/MyDrive/Spring2023/11-785 IDL"
# import required libraries
from pydub import AudioSegment
import pickle
import os

catpath = "/content/audio-retrieval-benchmark/sounddescs_data/categories.pkl"
descpath = "/content/audio-retrieval-benchmark/sounddescs_data/descriptions.pkl"

with open(catpath, "rb") as f:
    categories = pickle.load(f)

with open(descpath, "rb") as f:
    descriptions = pickle.load(f)

root = "/content/drive/MyDrive/Spring2023/11-785 IDL/soundescdata/audios"
out = "/content/drive/MyDrive/Spring2023/11-785 IDL/soundescdata/soundescs_clipped"

if not os.path.isdir(out):
    os.mkdir(out)

total_count = 0

cats = dict()
for clip_id in os.listdir(root):
    
    desc = descriptions[clip_id]
    audiopath = os.path.join(root, clip_id, f'{clip_id}.wav')

    for category in categories[clip_id]:
        if "Transport" == category:
            label = ""
            if "car" in desc.lower():
                label = "car"
            elif "aircraft" in desc.lower():
                label = "aircraft"
            elif "traffic" in desc.lower():
                label = "traffic"
            elif "tram" in desc.lower():
                label = "tram"
            elif "motorcycle" in desc.lower():
                label = "motorcycle"
            elif "train" in desc.lower():
                label = "train"
            elif "bus" in desc.lower():
                label = "bus"
            else:
                label = "transport"

        elif "Nature" == category:
            if "Rainforest" in desc:
                label = "rainforest"
            elif "Frog" in desc:
                label = "frog"
            elif "Mallard" in desc:
                label = "mallard"
            elif "Honey Bee" in desc:
                label = "honey bee"
            elif "Horse" in desc:
                label = "horse"
            elif "Waterfall" in desc:
                label = "waterfall"
            elif "Elephant" in desc:
                label = "elephant"
            else:
                label = "nature"
        else:
            label = category.lower()

        if label not in cats:
            cats[label] = 0
        cats[label] += 1

        audio = AudioSegment.from_wav(audiopath)

        for i in range (0, int(audio.duration_seconds), 10):
            t = i * 1000
            clip = audio[t: t + (10 * 1000)]
            outdir = os.path.join(out, label)

            if not os.path.isdir(outdir):
                os.mkdir(outdir)

            newpath = os.path.join(outdir, f"{category}_{cats[label]}.wav")
            print(f"Saved clip {clip_id} {i}:{i + 10} at {newpath}")
            clip.export(newpath, format="wav") 
            total_count += 1

print(cats)
print(sum(cats.values()), total_count)

In [ ]:
categories = ["car", "aircraft", "crowds", "footsteps", "clocks", "rainforest"]

## Create Dataset

In [ ]:
# Combine Files

import os
import random
from pydub import AudioSegment

# Define the directory where your dataset is located
dataset_dir = '/content/drive/MyDrive/Spring2023/11-785 IDL/soundescdata/soundescs_clipped'
output_dir = '/content/drive/MyDrive/Spring2023/11-785 IDL/soundescdata/outputs'

if not os.path.isdir(output_dir):
  os.mkdir(output_dir)

# Define the list of classes in your dataset
classes = ["car", "aircraft", "crowds", "footsteps", "clocks", "rainforest"]  # Add more class names as needed
print(classes)
# Define the duration of each 10s clip in milliseconds
clip_duration_ms = 10000

# Define the duration of each 2-minute clip in milliseconds
output_duration_ms = 120000

class_dirs = [os.path.join(dataset_dir, class_name) for class_name in classes]
audio_files = [[f for f in os.listdir(class_dir) if f.endswith('.wav')] for class_dir in class_dirs]

num_generate_clips = 1000
# Create x number of clipss
for j in range(num_generate_clips):
    # Create a list to store the class information
    class_info = []
    # Create an empty AudioSegment object to hold the combined clips from all classes
    combined_clip = AudioSegment.silent(duration=0)

    # Randomly select clips and combine them to the given clip
    for i in range(int(output_duration_ms/clip_duration_ms)):
        # Randomly select an audio file from the current class

        class_idx = random.randint(0, len(classes)-1)
        audio_file_idx = random.randint(0, len(audio_files[class_idx])-1)

        audio_file = audio_files[class_idx][audio_file_idx]



        # Load the selected audio file as an AudioSegment
        clip = AudioSegment.from_file(os.path.join(class_dirs[class_idx], audio_file))

        # Append the selected clip to the combined clip
        combined_clip = combined_clip + clip

        # Calculate the start and end times of the current class segment in milliseconds
        segment_start_time_ms = len(combined_clip) - len(clip)
        segment_end_time_ms = len(combined_clip)

        # Add the class information to the list
        class_info.append(f'{classes[class_idx]}\t{segment_start_time_ms}\t{segment_end_time_ms}')
        # Export the combined clip to a new audio file
    output_file = os.path.join(output_dir, f'{j}.wav')
    combined_clip.export(output_file, format='wav')
    print(f"Created {j}.wav!")
    # Write the class information to a text file
    output_txt_file = os.path.join(output_dir, f'{j}.txt')
    with open(output_txt_file, 'w') as f:
        f.write('\n'.join(class_info))
    print(f"Created {j}.txt!")


print(f'Successfully combined clips from different classes')
print(f'Successfully wrote class information')